In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Challenge: Make Your Network

Create a multi-layer perceptron neural network model to predict on a labeled dataset of your choosing. Compare this model to either a boosted tree or a random forest model and describe the relative tradeoffs between complexity and accuracy. Be sure to vary the hyperparameters of your MLP!

### Dataset: Predict Red Wine Quality
https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [35]:
df = pd.read_csv('red_wine_quality.csv', delimiter=';')

In [36]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [37]:
# Binarize 'high quality' as 7 or greater
df['top_quality'] = np.where(df['quality'] >=7, 1, 0)

# Determine class balance
print('Class balance:', df['top_quality'].mean())

# Drop integer quality value
df = df.drop('quality', 1)

Class balance: 0.1357098186366479


In [5]:
# Specify inputs
X = df.iloc[:, :11]
Y = df['top_quality']

### Neural Network

In [6]:
# Import model
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

In [7]:
# Establish and fit with single, 100 perceptron layer 
mlp = MLPClassifier(hidden_layer_sizes=(100,))
mlp.fit(X,Y)

# Cross validate and get scores
cross_val_score(mlp, X, Y, cv=5)

array([0.87538941, 0.85981308, 0.86520376, 0.86520376, 0.86520376])

In [8]:
# Try with 1000 perceptron layer
mlp = MLPClassifier(hidden_layer_sizes=(1000,))
mlp.fit(X,Y)

# Cross validate and get scores
cross_val_score(mlp, X, Y, cv=5)

array([0.86292835, 0.86292835, 0.86833856, 0.82131661, 0.86520376])

In [9]:
# 100 perceptrons has slighty better performance and much faster run
# time.  Try adding second layer using 100 perceptrons.
mlp = MLPClassifier(hidden_layer_sizes=(100, 10))
mlp.fit(X,Y)

# Cross validate and get scores
cross_val_score(mlp, X, Y, cv=5)

# Second layer introduces less consistency in cross validation

array([0.86292835, 0.85981308, 0.86520376, 0.85579937, 0.86206897])

In [10]:
# Adjust additional parameters using single, 100 perceptron layer 
mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='tanh',
    alpha=10, #wide range of values work.  Does not converge at 100
    learning_rate='adaptive' #adjusting doesn't make much diff
)
mlp.fit(X,Y)

# Cross validate and get scores
cross_val_score(mlp, X, Y, cv=5)

array([0.86292835, 0.86292835, 0.86520376, 0.86520376, 0.86520376])

Tweaking hyperparameters, accuracy of 86% as good as I can get.

### Random Forest

In [14]:
# Import model
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV

In [22]:
# Initialize model
forest = ensemble.RandomForestClassifier(
    max_features='sqrt',
)

# Set hyperparameter
max_depth = dict(max_depth = np.arange(1,10,1))

# Set up grid search for forest depth
grid = GridSearchCV(forest, max_depth, cv=5)

# Fit and get results
results = grid.fit(X, Y)
results.best_estimator_.get_params()['max_depth']

7

In [26]:
# Using 7 as max_depth results in worse performance than 4.  
# Go with 4.  Shorter trees better in forest anyways.
forest = ensemble.RandomForestClassifier(
    max_features='sqrt',
    max_depth=4
)

cross_val_score(forest, X, Y, cv=5)

array([0.86915888, 0.86292835, 0.88087774, 0.88087774, 0.88401254])

In [32]:
# Get features 
forest.fit(X,Y)

forest_features = pd.DataFrame()
forest_features['importance'] = forest.feature_importances_
forest_features['feature'] = X.columns

forest_features.sort_values('importance', ascending=False).head()

,importance,feature
10,0.293647,alcohol
1,0.170647,volatile acidity
9,0.138989,sulphates
4,0.076505,chlorides
7,0.074196,density


### Model Comparison
Roughly equal performance between both models in this instance.

Neural Network:
* More consistent performance across folds
* More computationally demanding
* No insights into feature importance

Random Forest:
* Slightly less consistent across folds
* Faster to run
* Gives feature importances

Is ANN supposed to be more accurate here?  Both seem pretty good.